SKRIPSI

# Import library

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk
import csv 
from nltk import ngrams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, validation_curve,RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from matplotlib_venn import venn2
from time import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics

np.random.seed(0)
stopwords = nltk.corpus.stopwords.words('english')
poster_Stemmer = nltk.PorterStemmer()


# Baca file csv

In [2]:
#baca csv
data_train_amazon = pd.read_csv('dataset/Amazon_Train.csv')
data_train_yelp = pd.read_csv('dataset/Yelp_Train.csv')
data_test_amazon = pd.read_csv('dataset/Amazon_Test.csv')
data_test_yelp = pd.read_csv('dataset/Yelp_Test.csv')

In [3]:
print(data_train_amazon.head())
print("\n")
print(data_train_yelp.head())

   Label                                           Sentimen
0      0  Buyer beware This is a self-published book, an...
1      0  The Worst! A complete waste of time. Typograph...
2      0  Oh please I guess you have to be a romance nov...
3      0  Awful beyond belief! I feel I have to write to...
4      0  Another Abysmal Digital Copy Rather than scrat...


   Label                                           Sentimen
0      0  I don't know what Dr. Goldberg was like before...
1      0  I'm writing this review to give you a heads up...
2      0  Owning a driving range inside the city limits ...
3      0  This place is absolute garbage...  Half of the...
4      0  Used to go there for tires, brakes, etc.  Thei...


In [4]:
print(data_test_amazon.head())
print("\n")
print(data_test_yelp.head())

   Label                                           Sentimen
0      0  Overly complicated Being both a U.S. history b...
1      0  Terrible Disappointment -- parts don't fit My ...
2      0  Didn't hold up.....very disappointed I bought ...
3      0  gene hates jezebel i love JLJ but this compila...
4      0  Nice toy but ... My six-year-old loves space a...


   Label                                           Sentimen
0      0  My wife and I used to love Arriba's, til recen...
1      0  You get what you pay for.  The food is inexpen...
2      0  Unfortunately, yesterday's visit was one of th...
3      0  I went into the Scottsdale location yesterday....
4      0  It takes a lot for me to write a review and bl...


# Preprocessing ke lowercase

In [5]:
iterator = 0
data_Preprocessing_Amazon_Train = data_train_amazon
data_Lowercase_Amazon_Train = []

data_Preprocessing_Yelp_Train = data_train_yelp
data_Lowercase_Yelp_Train = []

while iterator < len(data_train_amazon) :
    data_Lowercase_Amazon_Train.append(data_train_amazon.Sentimen[iterator].lower())
    iterator = iterator + 1

iterator = 0

while iterator < len(data_train_yelp) :
    data_Lowercase_Yelp_Train.append(data_train_yelp.Sentimen[iterator].lower())
    iterator = iterator + 1
    

In [6]:
iterator = 0
data_Preprocessing_Amazon_Test = data_test_amazon
data_Lowercase_Amazon_Test = []

data_Preprocessing_Yelp_Test = data_test_yelp
data_Lowercase_Yelp_Test = []

while iterator < len(data_test_amazon) :
    data_Lowercase_Amazon_Test.append(data_test_amazon.Sentimen[iterator].lower())
    iterator = iterator + 1

iterator = 0

while iterator < len(data_test_yelp) :
    data_Lowercase_Yelp_Test.append(data_test_yelp.Sentimen[iterator].lower())
    iterator = iterator + 1

In [7]:
data_Preprocessing_Amazon_Train['Lowercase'] = data_Lowercase_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an..."
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...


In [8]:
data_Preprocessing_Amazon_Test['Lowercase'] = data_Lowercase_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...


In [9]:
data_Preprocessing_Yelp_Train['Lowercase'] = data_Lowercase_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei..."


In [10]:
data_Preprocessing_Yelp_Test['Lowercase'] = data_Lowercase_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen..."
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th..."
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan angka

In [11]:
iterator = 0
data_RemoveNumber_Amazon_Train = []

data_RemoveNumber_Yelp_Train = []

while iterator < len(data_Preprocessing_Amazon_Train) :
    data_RemoveNumber_Amazon_Train.append(re.sub(r"\d+", "",data_Preprocessing_Amazon_Train.Lowercase[iterator]))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_RemoveNumber_Yelp_Train.append(re.sub(r"\d+", "",data_Preprocessing_Yelp_Train.Lowercase[iterator]))
    iterator = iterator + 1

In [12]:
iterator = 0
data_RemoveNumber_Amazon_Test = []

data_RemoveNumber_Yelp_Test = []

while iterator < len(data_Preprocessing_Amazon_Test) :
    data_RemoveNumber_Amazon_Test.append(re.sub(r"\d+", "",data_Preprocessing_Amazon_Test.Lowercase[iterator]))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_RemoveNumber_Yelp_Test.append(re.sub(r"\d+", "",data_Preprocessing_Yelp_Test.Lowercase[iterator]))
    iterator = iterator + 1

In [13]:
data_Preprocessing_Amazon_Train['RemoveNumber'] = data_RemoveNumber_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an..."
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [14]:
data_Preprocessing_Amazon_Test['RemoveNumber'] = data_RemoveNumber_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...


In [15]:
data_Preprocessing_Yelp_Train['RemoveNumber'] = data_RemoveNumber_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei..."


In [16]:
data_Preprocessing_Yelp_Test['RemoveNumber'] = data_RemoveNumber_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen..."
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th..."
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan tanda baca

In [17]:
iterator = 0
data_RemovePunctuation_Amazon_Train = []

data_RemovePunctuation_Yelp_Train = []

while iterator < len(data_Preprocessing_Amazon_Train) :
    data_RemovePunctuation_Amazon_Train.append(data_Preprocessing_Amazon_Train.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_RemovePunctuation_Yelp_Train.append(data_Preprocessing_Yelp_Train.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

In [18]:
iterator = 0
data_RemovePunctuation_Amazon_Test = []

data_RemovePunctuation_Yelp_Test = []

while iterator < len(data_Preprocessing_Amazon_Test) :
    data_RemovePunctuation_Amazon_Test.append(data_Preprocessing_Amazon_Test.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_RemovePunctuation_Yelp_Test.append(data_Preprocessing_Yelp_Test.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

In [19]:
data_Preprocessing_Amazon_Train['RemovePunctuation'] = data_RemovePunctuation_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an...",buyer beware this is a selfpublished book and ...
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...,the worst a complete waste of time typographic...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief i feel i have to write to ...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [20]:
data_Preprocessing_Amazon_Test['RemovePunctuation'] = data_RemovePunctuation_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...,overly complicated being both a us history buf...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...,terrible disappointment parts dont fit my son...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...,didnt hold upvery disappointed i bought this o...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but my sixyearold loves space and to...


In [21]:
data_Preprocessing_Yelp_Train['RemovePunctuation'] = data_RemovePunctuation_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...,i dont know what dr goldberg was like before ...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,im writing this review to give you a heads up ...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...,this place is absolute garbage half of the te...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei...",used to go there for tires brakes etc their p...


In [22]:
data_Preprocessing_Yelp_Test['RemovePunctuation'] = data_RemovePunctuation_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen...",my wife and i used to love arribas til recentl...
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for the food is inexpens...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...",unfortunately yesterdays visit was one of the ...
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday ...
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan Non alfabet

In [23]:
iterator = 0

data_Regex_alpabet_only_Amazon_Train = []

data_Regex_alpabet_only_Yelp_Train = []


while iterator < len(data_Preprocessing_Amazon_Train) :
    data_Regex_alpabet_only_Amazon_Train.append(" ".join(re.findall("[a-zA-Z]+", data_Preprocessing_Amazon_Train.RemovePunctuation[iterator])))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_Regex_alpabet_only_Yelp_Train.append(" ".join(re.findall(r"[a-zA-Z]+", data_Preprocessing_Yelp_Train.RemovePunctuation[iterator])))
    iterator = iterator + 1



In [24]:
iterator = 0

data_Regex_alpabet_only_Amazon_Test = []

data_Regex_alpabet_only_Yelp_Test = []


while iterator < len(data_Preprocessing_Amazon_Test) :
    data_Regex_alpabet_only_Amazon_Test.append(" ".join(re.findall("[a-zA-Z]+", data_Preprocessing_Amazon_Test.RemovePunctuation[iterator])))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_Regex_alpabet_only_Yelp_Test.append(" ".join(re.findall(r"[a-zA-Z]+", data_Preprocessing_Yelp_Test.RemovePunctuation[iterator])))
    iterator = iterator + 1

In [25]:
data_Preprocessing_Amazon_Train['Regex'] = data_Regex_alpabet_only_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an...",buyer beware this is a selfpublished book and ...,buyer beware this is a selfpublished book and ...
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...,the worst a complete waste of time typographic...,the worst a complete waste of time typographic...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief i feel i have to write to ...,awful beyond belief i feel i have to write to ...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [26]:
data_Preprocessing_Amazon_Test['Regex'] = data_Regex_alpabet_only_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...,overly complicated being both a us history buf...,overly complicated being both a us history buf...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...,terrible disappointment parts dont fit my son...,terrible disappointment parts dont fit my son ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...,didnt hold upvery disappointed i bought this o...,didnt hold upvery disappointed i bought this o...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but my sixyearold loves space and to...,nice toy but my sixyearold loves space and to ...


In [27]:
data_Preprocessing_Yelp_Train['Regex'] = data_Regex_alpabet_only_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...,i dont know what dr goldberg was like before ...,i dont know what dr goldberg was like before m...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,im writing this review to give you a heads up ...,im writing this review to give you a heads up ...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...,this place is absolute garbage half of the te...,this place is absolute garbage half of the tee...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei...",used to go there for tires brakes etc their p...,used to go there for tires brakes etc their pr...


In [28]:
data_Preprocessing_Yelp_Test['Regex'] = data_Regex_alpabet_only_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen...",my wife and i used to love arribas til recentl...,my wife and i used to love arribas til recentl...
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for the food is inexpens...,you get what you pay for the food is inexpensi...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...",unfortunately yesterdays visit was one of the ...,unfortunately yesterdays visit was one of the ...
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday ...,i went into the scottsdale location yesterday ...
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


In [29]:
Clean_Amazon_Sentiment_Train = data_Preprocessing_Amazon_Train['Regex']
Clean_Yelp_Sentiment_Train = data_Preprocessing_Yelp_Train['Regex']
Label_Amazon_Train = data_Preprocessing_Amazon_Train['Label']
Label_Yelp_Train = data_Preprocessing_Yelp_Train['Label']

Clean_Amazon_Sentiment_Test = data_Preprocessing_Amazon_Test['Regex']
Clean_Yelp_Sentiment_Test = data_Preprocessing_Yelp_Test['Regex']
Label_Amazon_Test = data_Preprocessing_Amazon_Test['Label']
Label_Yelp_Test = data_Preprocessing_Yelp_Test['Label']

In [30]:
# tf = tfidfvectorizer()
# tf.fit(train)
# trainx = tf.transform(train)
# testx = tf.transform(test)

vectorizer_Amazon = TfidfVectorizer(ngram_range=(2,3), analyzer = 'char')
vectorizer_Amazon.fit(Clean_Amazon_Sentiment_Train)
Data_Train_Amazon = vectorizer_Amazon.transform(Clean_Amazon_Sentiment_Train).toarray()
Vocabulary_Train_Amazon = vectorizer_Amazon.get_feature_names()
Data_Test_Amazon = vectorizer_Amazon.transform(Clean_Amazon_Sentiment_Test).toarray()

vectorizer_Yelp = TfidfVectorizer(ngram_range=(2,3),analyzer = 'char' )
vectorizer_Yelp.fit(Clean_Yelp_Sentiment_Train)
Data_Train_Yelp = vectorizer_Yelp.transform(Clean_Yelp_Sentiment_Train).toarray()
Vocabulary_Train_Yelp = vectorizer_Yelp.get_feature_names()
Data_Test_Yelp = vectorizer_Yelp.transform(Clean_Yelp_Sentiment_Test).toarray()

In [31]:
DataFrame_Amazon_Train = pd.DataFrame(Data_Train_Amazon, columns = Vocabulary_Train_Amazon)
print (DataFrame_Amazon_Train)

             a        a    aa        ab        ac        ad   ae        af  \
0     0.174588  0.080901  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
1     0.149335  0.047575  0.0  0.044579  0.052005  0.056086  0.0  0.000000   
2     0.122767  0.056888  0.0  0.026653  0.000000  0.000000  0.0  0.033833   
3     0.106905  0.041917  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
4     0.121334  0.023193  0.0  0.014488  0.016902  0.000000  0.0  0.000000   
...        ...       ...  ...       ...       ...       ...  ...       ...   
3745  0.134475  0.000000  0.0  0.035683  0.041627  0.000000  0.0  0.090590   
3746  0.093831  0.027593  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
3747  0.112662  0.028713  0.0  0.026905  0.000000  0.033850  0.0  0.034153   
3748  0.136468  0.041405  0.0  0.000000  0.000000  0.000000  0.0  0.019700   
3749  0.093960  0.059867  0.0  0.037398  0.000000  0.000000  0.0  0.000000   

            ag   ah  ...  zza  zzb  zze  zzf  zzi  zzl  zzs  zz

In [32]:
DataFrame_Amazon_Test = pd.DataFrame(Data_Test_Amazon, columns = Vocabulary_Train_Amazon)
print (DataFrame_Amazon_Test)

             a        a    aa        ab   ac        ad   ae        af  \
0     0.146314  0.062150  0.0  0.019412  0.0  0.048845  0.0  0.024641   
1     0.087126  0.013878  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2     0.091250  0.049835  0.0  0.015566  0.0  0.000000  0.0  0.039517   
3     0.089522  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
4     0.085220  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
...        ...       ...  ...       ...  ...       ...  ...       ...   
1245  0.170102  0.032514  0.0  0.020311  0.0  0.000000  0.0  0.000000   
1246  0.151306  0.044495  0.0  0.000000  0.0  0.069940  0.0  0.000000   
1247  0.097047  0.030917  0.0  0.000000  0.0  0.072896  0.0  0.000000   
1248  0.018945  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
1249  0.062201  0.019816  0.0  0.000000  0.0  0.000000  0.0  0.000000   

            ag   ah  ...  zza  zzb  zze  zzf  zzi  zzl  zzs  zzt  zzy  zzz  
0     0.000000  0.0  ...  0.0  0.0  0.0  0.0  

In [33]:
DataFrame_Yelp_Train = pd.DataFrame(Data_Train_Yelp, columns = Vocabulary_Train_Yelp)
print (DataFrame_Yelp_Train)

             a        a    aa        ab        ac        ad   ae        af  \
0     0.171153  0.047851  0.0  0.035851  0.000000  0.017313  0.0  0.014340   
1     0.140159  0.043810  0.0  0.012765  0.000000  0.036986  0.0  0.000000   
2     0.159834  0.054718  0.0  0.011957  0.045784  0.017323  0.0  0.000000   
3     0.190176  0.025319  0.0  0.022131  0.000000  0.000000  0.0  0.026556   
4     0.059505  0.035650  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
...        ...       ...  ...       ...       ...       ...  ...       ...   
3745  0.096302  0.020980  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
3746  0.100676  0.015079  0.0  0.000000  0.000000  0.000000  0.0  0.031631   
3747  0.080185  0.000000  0.0  0.033592  0.000000  0.000000  0.0  0.000000   
3748  0.147717  0.023086  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
3749  0.143471  0.053721  0.0  0.000000  0.000000  0.000000  0.0  0.000000   

           ag   ah  ...  zz   zza  zzb  zzc  zze  zzi  zzl  zzo

In [34]:
DataFrame_Yelp_Test = pd.DataFrame(Data_Test_Yelp, columns = Vocabulary_Train_Yelp)
print (DataFrame_Yelp_Test)

             a        a    aa        ab   ac        ad   ae        af  \
0     0.164639  0.026303  0.0  0.022991  0.0  0.000000  0.0  0.013794   
1     0.078701  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2     0.089317  0.045865  0.0  0.000000  0.0  0.000000  0.0  0.000000   
3     0.120057  0.009590  0.0  0.000000  0.0  0.000000  0.0  0.020117   
4     0.164111  0.023980  0.0  0.000000  0.0  0.000000  0.0  0.010061   
...        ...       ...  ...       ...  ...       ...  ...       ...   
1245  0.140150  0.083964  0.0  0.000000  0.0  0.035443  0.0  0.000000   
1246  0.073760  0.058919  0.0  0.000000  0.0  0.000000  0.0  0.000000   
1247  0.150321  0.045029  0.0  0.026239  0.0  0.000000  0.0  0.000000   
1248  0.021459  0.025712  0.0  0.000000  0.0  0.000000  0.0  0.000000   
1249  0.141502  0.036331  0.0  0.000000  0.0  0.000000  0.0  0.000000   

            ag   ah  ...  zz   zza  zzb  zzc  zze  zzi  zzl  zzo  zzy  zzz  
0     0.000000  0.0  ...  0.0  0.0  0.0  0.0  

In [35]:
Waktu_Training = time()
RF_Classifier_Amazon = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Amazon.fit(Data_Train_Amazon, Label_Amazon_Train)
print(f"\nWaktu Training Amazon: {round(time()-Waktu_Training, 3)}s")



Waktu Training Amazon: 22.192s


In [36]:
Waktu_Predict_Train = time()
Skor_Train_Amazon = RF_Classifier_Amazon.score(Data_Train_Amazon, Label_Amazon_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Amazon = RF_Classifier_Amazon.score(Data_Test_Amazon, Label_Amazon_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Amazon : {}".format(Skor_Train_Amazon))
print("Skor Random Forest Test Amazon : {}\n\n".format(Skor_Test_Amazon))
print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Amazon.predict(Data_Train_Amazon)
Confusion_matrix = confusion_matrix(Label_Amazon_Train, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy Train: ", accuracy_score(Label_Amazon_Train, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Amazon_Train, RFC_predict))

print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Amazon.predict(Data_Test_Amazon)
Confusion_matrix = confusion_matrix(Label_Amazon_Test, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy TEST: ", accuracy_score(Label_Amazon_Test, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Amazon_Test, RFC_predict))

waktu prediksi (train): 0.442s
waktu prediksi (test): 0.177s

Skor Random Forest Train Amazon : 0.8981333333333333
Skor Random Forest Test Amazon : 0.8168


----------------------------------------------------


[[1690  185]
 [ 197 1678]]

Accuracy Train:  0.8981333333333333


              precision    recall  f1-score   support

           0       0.90      0.90      0.90      1875
           1       0.90      0.89      0.90      1875

    accuracy                           0.90      3750
   macro avg       0.90      0.90      0.90      3750
weighted avg       0.90      0.90      0.90      3750

----------------------------------------------------


[[502 123]
 [106 519]]

Accuracy TEST:  0.8168


              precision    recall  f1-score   support

           0       0.83      0.80      0.81       625
           1       0.81      0.83      0.82       625

    accuracy                           0.82      1250
   macro avg       0.82      0.82      0.82      1250
weighted avg       

In [37]:
Waktu_Training = time()
RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Yelp.fit(Data_Train_Yelp, Label_Yelp_Train)
print(f"\nWaktu Training Yelp: {round(time()-Waktu_Training, 3)}s")


Waktu Training Yelp: 23.305s


In [38]:
Waktu_Predict_Train = time()
Skor_Train_Yelp = RF_Classifier_Yelp.score(Data_Train_Yelp, Label_Yelp_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Yelp = RF_Classifier_Yelp.score(Data_Test_Yelp, Label_Yelp_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Yelp : {}".format(Skor_Train_Yelp))
print("Skor Random Forest Test Yelp : {}".format(Skor_Test_Yelp))
print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Yelp.predict(Data_Train_Yelp)
Confusion_matrix = confusion_matrix(Label_Yelp_Train, RFC_predict)
print("\n\n")
print(Confusion_matrix)
print("\nAccuracy Train: ", accuracy_score(Label_Yelp_Train, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Yelp_Train, RFC_predict))

print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Yelp.predict(Data_Test_Yelp)
Confusion_matrix = confusion_matrix(Label_Yelp_Test, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy Test: ", accuracy_score(Label_Yelp_Test, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Yelp_Test, RFC_predict))

waktu prediksi (train): 0.446s
waktu prediksi (test): 0.171s

Skor Random Forest Train Yelp : 0.9026666666666666
Skor Random Forest Test Yelp : 0.8552
----------------------------------------------------





[[1649  226]
 [ 139 1736]]

Accuracy Train:  0.9026666666666666


              precision    recall  f1-score   support

           0       0.92      0.88      0.90      1875
           1       0.88      0.93      0.90      1875

    accuracy                           0.90      3750
   macro avg       0.90      0.90      0.90      3750
weighted avg       0.90      0.90      0.90      3750

----------------------------------------------------


[[533  92]
 [ 89 536]]

Accuracy Test:  0.8552


              precision    recall  f1-score   support

           0       0.86      0.85      0.85       625
           1       0.85      0.86      0.86       625

    accuracy                           0.86      1250
   macro avg       0.86      0.86      0.86      1250
weighted avg       0.8

In [ ]:
Vocabulary_Importance_Amazon = []
iterator = 0
length = len (Vocabulary_Train_Amazon)

while iterator < length : 
    if RF_Classifier_Amazon.feature_importances_[iterator] > 0 :
        Vocabulary_Importance_Amazon.append(Vocabulary_Train_Amazon[iterator])
    iterator = iterator + 1

# Membuat Vectorizer baru dari data fitur importance only

In [ ]:
vectorizer_importance_amazon = TfidfVectorizer(vocabulary = Vocabulary_Importance_Amazon, ngram_range=(2,3),analyzer = 'char')

vectorizer_importance_amazon.fit(Clean_Amazon_Sentiment_Train)
Vocabulary_Importance_Amazon_Train = vectorizer_importance_amazon.get_feature_names()

Data_Train_Importance_Amazon = vectorizer_importance_amazon.transform(Clean_Amazon_Sentiment_Train).toarray()
Data_Test_Importance_Amazon = vectorizer_importance_amazon.transform(Clean_Amazon_Sentiment_Test).toarray()

# tf = tfidfvectorizer()
# tf.fit(train)
# trainx = tf.transform(train)
# testx = tf.transform(test)

In [ ]:
DataFrame_Importance_Amazon = pd.DataFrame(Data_Train_Importance_Amazon, columns = Vocabulary_Importance_Amazon_Train)
print (DataFrame_Importance_Amazon)

In [ ]:
DataFrame_Importance_Amazon_Test = pd.DataFrame(Data_Test_Importance_Amazon, columns = Vocabulary_Importance_Amazon_Train)
print (DataFrame_Importance_Amazon_Test)

In [ ]:
Waktu_Training = time()
RF_Classifier_Amazon_Importance = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Amazon_Importance.fit(Data_Train_Importance_Amazon, Label_Amazon_Train)
print(f"\nWaktu Training Amazon Feature Importance Only: {round(time()-Waktu_Training, 3)}s")

In [ ]:
Waktu_Predict_Train = time()
Skor_Train_Amazon_FI = RF_Classifier_Amazon_Importance.score(Data_Train_Importance_Amazon, Label_Amazon_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Amazon_FI = RF_Classifier_Amazon_Importance.score(Data_Test_Importance_Amazon, Label_Amazon_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Amazon Feature Importance : {}".format(Skor_Train_Amazon_FI))
print("Skor Random Forest Test Amazon Feature Importance : {}".format(Skor_Test_Amazon_FI))

print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Amazon_Importance.predict(Data_Train_Importance_Amazon)
Confusion_matrix = confusion_matrix(Label_Amazon_Train, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy Train: ", accuracy_score(Label_Amazon_Train, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Amazon_Train, RFC_predict))

print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Amazon_Importance.predict(Data_Test_Importance_Amazon)
Confusion_matrix = confusion_matrix(Label_Amazon_Test, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy Test: ", accuracy_score(Label_Amazon_Test, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Amazon_Test, RFC_predict))

# Mengetahui akurasi dengan hanya mengambil data train dari interseksi antara data yelp dan amazon(Feature imporantace > 0)

In [ ]:
Vocabulary_intersection = list(set(
                            Vocabulary_Importance_Amazon) & set (
                            Vocabulary_Train_Yelp))

In [ ]:
venn2(subsets = (len(Vocabulary_Importance_Amazon_Train), len(Vocabulary_Train_Yelp), len(Vocabulary_intersection)), set_labels = ('Fitur Model acuan yang fiture importance > 0', 'Vocabulary Model baru'))

In [ ]:
vectorizer_intersection_yelp = TfidfVectorizer(vocabulary = Vocabulary_intersection, ngram_range=(2,3),analyzer = 'char')
vectorizer_intersection_yelp.fit(Clean_Yelp_Sentiment_Train)

Vocabulary_Intersection_Train = vectorizer_intersection_yelp.get_feature_names()
Data_Train_Intersection = vectorizer_intersection_yelp.transform(Clean_Yelp_Sentiment_Train).toarray()
Data_Test_Intersection = vectorizer_intersection_yelp.transform(Clean_Yelp_Sentiment_Test).toarray()

# tf = tfidfvectorizer()
# tf.fit(train)
# trainx = tf.transform(train)
# testx = tf.transform(test)

In [ ]:
DataFrame_Intersection_Train = pd.DataFrame(Data_Train_Intersection, columns = Vocabulary_Intersection_Train)
print(DataFrame_Intersection_Train)

# Akurasi interseksi yelp dan amazon (FI)

In [ ]:
waktu_training_interseksi = time()
RF_Classifier_Yelp_intersec = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Yelp_intersec.fit(Data_Train_Intersection, Label_Yelp_Train)
print(f"\nWaktu Training data interseksi Only: {round(time()-waktu_training_interseksi, 3)}s")

In [ ]:
Waktu_Predict_Train = time()
Skor_Train_Interseksi_Yelp = RF_Classifier_Yelp_intersec.score(Data_Train_Intersection, Label_Yelp_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Interseksi_Yelp = RF_Classifier_Yelp_intersec.score(Data_Test_Intersection, Label_Yelp_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Interseksi : {}".format(Skor_Train_Interseksi_Yelp))
print("Skor Random Forest Test Interseksi : {}".format(Skor_Test_Interseksi_Yelp))
print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Yelp_intersec.predict(Data_Train_Intersection)
Confusion_matrix = confusion_matrix(Label_Yelp_Train, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy Train: ", accuracy_score(Label_Yelp_Train, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Yelp_Train, RFC_predict))

print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Yelp_intersec.predict(Data_Test_Intersection)
Confusion_matrix = confusion_matrix(Label_Yelp_Test, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy Test: ", accuracy_score(Label_Yelp_Test, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Yelp_Test, RFC_predict))

# Mengetahui akurasi dengan hanya mengambil Data Train Yelp yang diseleksi dengan data amazon feature importance > 0

In [ ]:
# Meseleksi jika terdapat term yang sama pada dictionary yelp dan amazon maka akan di cek apakah term tersebut, 
# memiliki feature importance > 0, jika tidak maka tidak akan dimasukan untuk data train selanjutnya  

In [ ]:
Vocabulary_Seleksi_Yelp = []
iterator = 0
length = len(Vocabulary_Train_Yelp)

while iterator < length : 
#     jika interseksi dengan amazon dan bukan fitur importance maka vocab tidak dimasukan
    if  (Vocabulary_Train_Yelp[iterator] in Vocabulary_Train_Amazon) and (Vocabulary_Train_Yelp[iterator] not in Vocabulary_Importance_Amazon_Train) : 
        iterator = iterator + 1
        continue
        
    Vocabulary_Seleksi_Yelp.append(Vocabulary_Train_Yelp[iterator])
    iterator = iterator + 1

In [ ]:
vectorizer_seleksi_yelp = TfidfVectorizer(vocabulary = Vocabulary_Seleksi_Yelp, ngram_range=(2,3),analyzer = 'char')
vectorizer_seleksi_yelp.fit(Clean_Yelp_Sentiment_Train)

Vocabulary_Seleksi_Yelp_Train = vectorizer_seleksi_yelp.get_feature_names()
Data_Train_Seleksi_Yelp = vectorizer_seleksi_yelp.transform(Clean_Yelp_Sentiment_Train).toarray()
Data_Test_Seleksi_Yelp = vectorizer_seleksi_yelp.transform(Clean_Yelp_Sentiment_Test).toarray()

# tf = tfidfvectorizer()
# tf.fit(train)
# trainx = tf.transform(train)
# testx = tf.transform(test)

In [ ]:
DataFrame_Seleksi = pd.DataFrame(Data_Train_Seleksi_Yelp, columns = Vocabulary_Seleksi_Yelp_Train)
print (DataFrame_Seleksi)

In [ ]:
waktu_training_seleksi_yelp = time()
RF_Classifier_Yelp_Seleksi = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Yelp_Seleksi.fit(Data_Train_Seleksi_Yelp, Label_Yelp_Train)
print(f"\nWaktu Training data seleksi: {round(time()-waktu_training_seleksi_yelp, 3)}s")


In [ ]:
Waktu_Predict_Train = time()
Skor_Train_Seleksi_Yelp = RF_Classifier_Yelp_Seleksi.score(Data_Train_Seleksi_Yelp, Label_Yelp_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Seleksi_Yelp = RF_Classifier_Yelp_Seleksi.score(Data_Test_Seleksi_Yelp, Label_Yelp_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Seleksi : {}".format(Skor_Train_Seleksi_Yelp))
print("Skor Random Forest Test Seleksi : {}".format(Skor_Test_Seleksi_Yelp))

print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Yelp_Seleksi.predict(Data_Train_Seleksi_Yelp)
Confusion_matrix = confusion_matrix(Label_Yelp_Train, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy Test: ", accuracy_score(Label_Yelp_Train, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Yelp_Train, RFC_predict))

print("----------------------------------------------------\n\n")
RFC_predict = RF_Classifier_Yelp_Seleksi.predict(Data_Test_Seleksi_Yelp)
Confusion_matrix = confusion_matrix(Label_Yelp_Test, RFC_predict)
print(Confusion_matrix)
print("\nAccuracy TEST: ", accuracy_score(Label_Yelp_Test, RFC_predict))
print("\n")
print(metrics.classification_report(Label_Yelp_Test, RFC_predict))

In [ ]:
from joblib import dump, load

In [ ]:
dump(RF_Classifier_Yelp_Seleksi, 'RF_seleksi.joblib')

In [ ]:
from joblib import dump, load
dump(vectorizer_seleksi_yelp, 'vectroizer_seleksi.joblib')

In [ ]:
data_train_tes = pd.read_csv('tes.csv')

In [ ]:
print(data_train_tes.head())

In [ ]:
dfprint = pd.DataFrame()
dfprint['COMMENT'] = data_train_tes.sentiment

In [ ]:
text_baru = dfprint['COMMENT'].astype(str)    
text_baru = text_baru.apply(lambda x: x.lower()) #Lower Case
text_baru = text_baru.apply(lambda x: re.sub(r"\d", "", x)) #Remove Number    
text_baru = text_baru.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #punctuation  
text_baru = text_baru.apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))   

In [ ]:
print(text_baru[0])

In [ ]:
print (text_baru[0])

In [ ]:
print(data_train_tes.sentiment[0])

In [ ]:
cols = "``".join([str(i) for i in data_train_tes.columns.tolist()])

In [ ]:
print(cols)

In [ ]:
print(data_train_tes[cols][0])